# Automated UAS SkyWatch Standard Format Remark Functions & Script

## Origin Location: ICAO Airport Identifier
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 19, 2021

***
***How to Use:***

`To obtain the Standard Format Remarks along with complete UAS Location information in the form of latitudes and longitudes, call: uas_lat_long('file_path')`

`This function will output the dataframe containing the complete UAS location information for Standard Format Remarks referencing an ICAO Airport Identifier.`
***

## Import Statements

In [21]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

## Read in Files

In [22]:
# Read in Incidents_Cleaned_Standard.csv (contains 5191 records that are of 'Standard Format')
faa_standard = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Incidents_Cleaned_Standard_NoDuplicates.csv')

# REMARK MUST REFERENCE AN ICAO AIRPORT IDENTIFIER

## uas_loc_airportICAO()

Create a Function that takes in a .csv file of SkyWatch records for which the Remarks are in the Standard Format and uses Regular Expressions to extract the portion of the REMARK that reads 'XX NM direction [of] XXX' from the remark where XXX references an ICAO AIRPORT IDENTIFIER

In [23]:
# Create Function FOR STANDARD FORMAT REMARKS that:
# A. grabs REMARKS field from input dataframe/file.csv containing STANDARD FORMAT REMARKS
# B. Uses Regular Expressions to extract 'XX NM direction [of] XXX' from the remark

def uas_loc_airportICAO (file):
    '''
    This function takes in a .csv file of SkyWatch records for which the Remarks are in the Standard Format and uses
    regular expressions to extract the portion of the remark that reads 'XX NM direction [of] XXX'.
    
    This function also exports the following .csv files:
    A) StandardRemark_UAS_Location_ICAO_NULL -- Contains all UAS Locations from Standard Format Remarks (INCLUDING NULL VALUES)
    B) StandardRemark_UAS_Location_ICAO_nonNULL -- Contains all UAS Locations from Standard Format Remarks (NOT INCLUDING NULL VALUES)
    
    @param file: the file path (str) for a .csv file containing SkyWatch reports for which the REMARKS are
                    in Standard Format
    
    @output df1 : a dataframe containing the Standard Format REMARK as well as the UAS Location portion of the remark
                that reads 'XX NM direction [of] XXX' as well as the associated Airport information linked with the 
                Airport's FAA Identifier
    '''
    # Read in .csv file provided by user
    faa_standard = pd.read_csv(file)
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = faa_standard['REMARKS']

    # regular expression for any heading/direction
    headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?NM* [N|S|E|W|NW|NE|SW|SE|SSE|SSW|SNE|SNW|NNE|NSE|NNE|NNW|WSW|WNW|WSE|WNE|ENE|ESE|ESW|ENW]*\s?of?\s?[A-Z][A-Z][A-Z][A-Z]?'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        head_dir = re.findall(headir_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(head_dir)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    remark_uas_loc_df = pd.DataFrame()
    remark_uas_loc_df['REMARKS'] = remark
    remark_uas_loc_df['UAS Location'] = uas_loc
    
    # Export final dataframe (INCLUDING NULL UAS LOCATIONS) to .csv file
    remark_uas_loc_df.to_csv('StandardRemark_UAS_Location_ICAO_NULL.csv', index = False)
    
    # Get list of UAS locations from the above dataframe
    uas_loc = remark_uas_loc_df['UAS Location'] 

    # If the regular expressions did not hit on any location information, pass it UNKN for now
    for i in range(len(remark_uas_loc_df)):
        if len(uas_loc[i]) == 0:
            uas_loc[i] = 'UNKN'
    
    uas_loc_nonNull = uas_loc[uas_loc != 'UNKN'].to_list()
    uas_airport = []

    for i in range(len(uas_loc_nonNull)):
        airport = uas_loc_nonNull[i][0].split(' ')[-1]

        if len(airport) <= 4:
            uas_airport.append(airport)
        else:
            trim_air = airport[-3:]
            uas_airport.append(trim_air) # we perform this trim to account for a lack of space, for example: 'ofCLE'
    
    uas_airport_df = pd.DataFrame()
    uas_airport_df['ICAO_ID'] = uas_airport
    
    # Create Dataframe of Standard Format Remarks with NON NULL UAS Locations
    remark_uas_loc_nn = remark_uas_loc_df[remark_uas_loc_df['UAS Location'] != 'UNKN'].reset_index()
    
    # Export final dataframe (EXCLUDING NULL UAS LOCATIONS) to .csv file
    remark_uas_loc_nn.to_csv('StandardRemark_UAS_Location_ICAO_nonNULL.csv', index = False)
    
    # JOIN DATASETS TO HAVE ALL LOCATION INFORMATION FROM REMARK

    # DATAFRAMES USED:
    # airportsC = airports_cleaned.csv (the cleaned airports dataset)
    # uas_airport_df = dataframe of FAA Airport Identifiers found in the 3033 Standard Formats hit on by Regular Expressions
    # full_loc = dataframe containing the UAS sighting remark from SkyWatch as well as the UAS location information extracted

    uas_air_loc= pd.merge(uas_airport_df, airportsC, on='ICAO_ID', how='left')
    full_loc = pd.DataFrame()
    full_loc['REMARKS'] = remark_uas_loc_nn['REMARKS']
    full_loc['UAS_LOC'] = remark_uas_loc_nn['UAS Location']

    df1 = pd.concat([full_loc, uas_air_loc], axis = 1) #dataframe with final UAS Location information READY FOR CALCULATIONS
                                                                # from the 3,033 records extracted from the Standard Format Remarks
    
    df1 = df1.dropna().reset_index() # DROP ALL values that do not reference an airport using FAA Identifier
    
    return df1

In [24]:
uas_loc_airportICAO('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Incidents_Cleaned_Standard_NoDuplicates.csv')

,index,REMARKS,UAS_LOC,ICAO_ID,GLOBAL_ID,IDENT,NAME,LATITUDE,LONGITUDE
0,361,Aircraft observed a gray UAS off the left side...,[6 NM SSE of PHOG],PHOG,C65438E4-5646-489A-910D-EBD48F5EDFF2,OGG,Kahului,20.898649,-156.430459
1,608,Aircraft observed a white quad-copter UAS at 1...,[12 NM NE of KCLR],KCLR,E17429AA-BAE5-43DB-8864-25BB5EB0C231,CLR,Cliff Hatfield Meml,33.131333,-115.521389
2,633,Aircraft observed a UAS 200 feet below while S...,[1NM NW of TJIG],TJIG,971CDC84-BD79-416E-8C79-F7FDBAC807D4,SIG,Fernando Luis Ribas Dominicci,18.456778,-66.098472
3,789,Aircraft observed a UAS pass 50 feet above whi...,[2 NM NW of PABE],PABE,C6ED3B28-0D5A-42D2-B3EC-4F17BE9A60FE,BET,Bethel,60.778556,-161.837167
4,2134,Aircraft observed an orange UAS while E bound ...,[7 NM W of TJSJ],TJSJ,E01E1299-FCD2-48CD-81EB-0A91D0064B0A,SJU,Luis Munoz Marin Intl,18.439399,-66.002133
5,2477,Aircraft observed a black and gray UAS while E...,[3 NM NW of TJSJ],TJSJ,E01E1299-FCD2-48CD-81EB-0A91D0064B0A,SJU,Luis Munoz Marin Intl,18.439399,-66.002133


## bearing_dist_ICAOIdent()

Create a Function that takes in a .csv file of SkyWatch records for which the Remarks are in the Standard Format and uses the uas_loc_airportICAO() function to split UAS Locations extracted from Remarks into their origin identifier, distance (in both NM and Kilometers) and bearing (as an abbreviation and in degrees).

In [25]:
def bearing_dist_ICAOIdent(file):
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the uas_loc_airportICAO() function to split the UAS Location within the remark into the distance (in both NM and 
    kilometers), bearing abbreviation and degrees, and associated ICAO airport Identifier.
    
    @param file : the file path (str) for a .csv file containing SkyWatch reports for which the REMARKS are
                    in Standard Format
    @output new_df_validBearing: returns a dataframe containing the Remark, UAS Location, ICAO Airport Identifier, Distance 
                                    (in NM) from the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks with invalid bearing information is 
                    exported into the StandardRemark_UAS_Location_Converted_ICAOallBearing.csv file 
                    
                *** it should be noted that the dataset that only contains remarks with VALID bearing information is
                    exported into the StandardRemark_UAS_Location_General_ICAOvalidBearing.csv file
    '''
    # Call uas_loc_airportFAA() function created above
    df1 = uas_loc_airportICAO(file)
    
    # Create Dictionary of Bearings and Respective Degrees
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    # Extract Distances and Bearings from UAS_LOC that were pulled from Remarks
    # with a Standard Format in SkyWatch
    uas_loc = df1['UAS_LOC']
    distances = []
    bearings = []

    for i in range(len(uas_loc)):
        split = uas_loc[i][0].split(' ')[0:3]

        if 'NM' in split[0]: #if there is no space between the distance and NM (i.e. 3NM)
            distance_nm = split[0][0:len(split[0]) - 2]
            bearing_abrev = split[1].replace('of', '')

        else: 
            distance_nm = split[0]
            bearing_abrev = split[2].replace('of', '')

        distances.append(float(distance_nm))
        bearings.append(bearing_abrev)
    
    # Create DataFrame to store UAS Location Information
    new_df = pd.DataFrame(df1['REMARKS'])
    new_df['UAS_LOC'] = df1['UAS_LOC']
    new_df['IDENT'] = df1['IDENT']
    new_df['ICAO_ID'] = df1['ICAO_ID']
    new_df['Distance_NM'] = distances
    new_df['Bearing'] = bearings
    
    # Convert all distance from NM to kilometers and all bearing abbreviations to their associated degrees
    dist_kilo = []

    for i in range(len(new_df)):
        distanceKilo = new_df['Distance_NM'][i] * 1.852 # converting NM to kilometers
        dist_kilo.append(distanceKilo)

    bearDegree = pd.DataFrame(new_df['Bearing'])
    bearDegree = bearDegree.replace({'Bearing': bearing_deg})
    
    for i in range(len(bearDegree)):
        if bearDegree['Bearing'][i] == '':
            bearDegree.replace('', np.nan, inplace = True)

    new_df['Distance_Kilometers'] = dist_kilo
    new_df['Bearing_Degrees'] = bearDegree
    
    new_df['Airport_Latitude'] = df1['LATITUDE']
    new_df['Airport_Longitude'] = df1['LONGITUDE']
    
    # Export final new_df to .csv 
    new_df.to_csv('StandardRemark_UAS_Location_Converted_ICAOallBearing.csv', index = False)
    
    # Create Dataframe of Standard Format Remarks without NULL BEARING INFO
    new_df_validBearing = new_df.dropna().reset_index()
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    new_df_validBearing.to_csv('StandardRemark_UAS_Location_General_ICAOvalidBearing.csv', index = False)
    
    return new_df_validBearing

In [27]:
bearing_dist_ICAOIdent('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Incidents_Cleaned_Standard_NoDuplicates.csv')

,index,REMARKS,UAS_LOC,IDENT,ICAO_ID,Distance_NM,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude
0,0,Aircraft observed a gray UAS off the left side...,[6 NM SSE of PHOG],OGG,PHOG,6.0,SSE,11.112,158,20.898649,-156.430459
1,1,Aircraft observed a white quad-copter UAS at 1...,[12 NM NE of KCLR],CLR,KCLR,12.0,NE,22.224,45,33.131333,-115.521389
2,2,Aircraft observed a UAS 200 feet below while S...,[1NM NW of TJIG],SIG,TJIG,1.0,NW,1.852,315,18.456778,-66.098472
3,3,Aircraft observed a UAS pass 50 feet above whi...,[2 NM NW of PABE],BET,PABE,2.0,NW,3.704,315,60.778556,-161.837167
4,4,Aircraft observed an orange UAS while E bound ...,[7 NM W of TJSJ],SJU,TJSJ,7.0,W,12.964,270,18.439399,-66.002133
5,5,Aircraft observed a black and gray UAS while E...,[3 NM NW of TJSJ],SJU,TJSJ,3.0,NW,5.556,315,18.439399,-66.002133


## uas_ICAO_lat_long()

Calculates the latitude and longitude for the sighted UAS using the geopy library (***WHEN THE UAS SIGHTING REFERENCES AN ICAO AIRPORT IDENTIFIER***)

In [28]:
def uas_ICAO_lat_long(file):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param file : the file path (str) for a .csv file containing SkyWatch reports for which the REMARKS are
                    in Standard Format
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, ICAO Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_ICAOairIdent.csv file.
                    
    
    '''
    # Call bearing_dist_ICAOIdent() function created above
    new_df = bearing_dist_ICAOIdent(file)
    
    uas_lat = []
    uas_long = []


    for i in range(len(new_df)):
        lat_airport = pd.to_numeric(new_df['Airport_Latitude'][i])
        long_airport = pd.to_numeric(new_df['Airport_Longitude'][i])
        b = pd.to_numeric(new_df['Bearing_Degrees'][i])
        d = pd.to_numeric(new_df['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2, = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
    
    # Append UAS Lat/Long information to DataFrame
    new_df['UAS_Latitude'] = uas_lat
    new_df['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    new_df.to_csv('StandardRemark_UAS_LatLong_validBearing_ICAOairIdent.csv', index = False)
    
    return new_df

In [30]:
uas_ICAO_lat_long('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Incidents_Cleaned_Standard_NoDuplicates.csv')

,index,REMARKS,UAS_LOC,IDENT,ICAO_ID,Distance_NM,Bearing,Distance_Kilometers,Bearing_Degrees,Airport_Latitude,Airport_Longitude,UAS_Latitude,UAS_Longitude
0,0,Aircraft observed a gray UAS off the left side...,[6 NM SSE of PHOG],OGG,PHOG,6.0,SSE,11.112,158,20.898649,-156.430459,20.805586,-156.390474
1,1,Aircraft observed a white quad-copter UAS at 1...,[12 NM NE of KCLR],CLR,KCLR,12.0,NE,22.224,45,33.131333,-115.521389,33.272911,-115.352712
2,2,Aircraft observed a UAS 200 feet below while S...,[1NM NW of TJIG],SIG,TJIG,1.0,NW,1.852,315,18.456778,-66.098472,18.468609,-66.110871
3,3,Aircraft observed a UAS pass 50 feet above whi...,[2 NM NW of PABE],BET,PABE,2.0,NW,3.704,315,60.778556,-161.837167,60.802053,-161.885274
4,4,Aircraft observed an orange UAS while E bound ...,[7 NM W of TJSJ],SJU,TJSJ,7.0,W,12.964,270,18.439399,-66.002133,18.439359,-66.124852
5,5,Aircraft observed a black and gray UAS while E...,[3 NM NW of TJSJ],SJU,TJSJ,3.0,NW,5.556,315,18.439399,-66.002133,18.474890,-66.039330
